In [ ]:
# Import des bibliothèques nécessaires
import sys
import os
sys.path.append('../src')

import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# Import de nos modules personnalisés
from data_utils import DataManager
from model_architecture import ImageClassifierBuilder, create_data_augmentation_layer, create_preprocessing_layer

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Configuration pour les graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [ ]:
# Initialisation du DataManager
data_manager = DataManager(data_dir="../data")

# Vérification et téléchargement du dataset si nécessaire
if not data_manager.verify_dataset_structure():
    print("Dataset non trouvé. Tentative de téléchargement...")
    success = data_manager.download_dataset()
    if not success:
        print("⚠️ Échec du téléchargement automatique.")
        print("Veuillez télécharger manuellement le dataset depuis:")
        print("https://www.kaggle.com/datasets/anthonytherrien/image-classification-dataset-32-classes")
        print("Et l'extraire dans le dossier '../data/raw/'")
else:
    print("✅ Dataset trouvé et vérifié!")

# Obtenir les informations sur le dataset
dataset_info = data_manager.get_dataset_info()
print(f"\n📊 Informations sur le dataset:")
print(f"Nombre de classes: {dataset_info['num_classes']}")
print(f"Nombre total d'images: {dataset_info['total_images']}")
print(f"Classes: {dataset_info['class_names'][:10]}...")  # Afficher les 10 premières classes
